# Case study Starbucks satisfaction survey. Inferential statistics.
Por: Francisco Jose Diaz **-** Data analyst de El Salvador

Este notebook tiene como objetivo practicar estadistica inferencial con las bibliotecas de Python para construir intervalos de confianza para la diferencia entre dos proporciones y medias de poblacion. Los resultados son con fines didacticos.

Fuente de los datos: [Kaggle](https://www.kaggle.com/datasets/mahirahmzh/starbucks-customer-retention-malaysia-survey?select=Starbucks+satisfactory+survey+encode+cleaned.csv)

## Setup

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm

In [2]:
df = pd.read_csv("Starbucks satisfactory survey encode cleaned.csv")

# Explorando los datos

In [3]:
df.head()

,Id,gender,age,status,income,visitNo,method,timeSpend,location,membershipCard,...,chooseRate,promoMethodApp,promoMethodSoc,promoMethodEmail,promoMethodDeal,promoMethodFriend,promoMethodDisplay,promoMethodBillboard,promoMethodOthers,loyal
0,1,1,1,0,0,3,0,1,0,0,...,3,1,1,1,1,1,1,1,1,0
1,2,1,1,0,0,3,2,0,1,0,...,2,1,1,1,1,1,1,1,1,0
2,3,0,1,2,0,2,0,1,2,0,...,3,1,1,1,1,1,1,1,1,0
3,4,1,1,0,0,3,2,0,2,1,...,3,1,1,1,1,1,1,1,1,1
4,5,0,1,0,0,2,2,1,1,1,...,3,1,1,1,1,1,1,1,1,0


# Confidence intervals
Los intervalos de confianza son un rango calculado alrededor de un parametro que se respalda con un cierto nivel de confianza.

Supongamos que el gerente de marketing nos pide el dato del promedio de ingreso de los consumidores de Starbucks, entonces para presentarle un resultado con fundamento estadistico construiremos un intervalo de confianza.

La encuesta nos presenta la variable income pero esta codificada. Para hacer mas entretenido este notebook vamos a modificar esa variable. Se tiene en cuenta que este notebook es todo con fines didacticos para practicar estadistica inferencial. 

In [4]:
# usamos libreria statsmodels
import statsmodels.api as sm

# usaremos la libreria random
from random import randrange

Modicamos la variable

In [5]:
df["incomeNumber"]=df.income.replace({0:randrange(1000,4999), 1:randrange(5000,9999), 2:randrange(10000,19999), 3:randrange(20000,29999), 3:randrange(30000,45000)})

In [6]:
sm.stats.DescrStatsW(df["incomeNumber"]).zconfint_mean()

(4472.812160898447, 7239.6834143227925)

Con 95% de confianza se tiene la certeza que el ingreso de los consumidores se encuentra en un rango de 4472.81 a 7239.68 dolares anuales.

# Confidence intervals for one proportion

El objetivo de esta seccion es demostrar la construccion de intervalos de confianza para una proporcion. Primero reemplazamos los codigos numericos en las variables de interes con etiquetas de texto, con esto ahora podemos tabular los numeros de female y male yes-menbershipcard and non-menbershipcard:

In [7]:
df["genderx"]=df.gender.replace({0:"Male", 1:"Female"})

In [8]:
df["membershipCardx"]=df.membershipCard.replace({0:"Yes",1:"No"})

In [9]:
dx = df[["membershipCardx", "genderx"]].dropna()  # dropna drops cases where either variable is missing
pd.crosstab(dx.membershipCardx, dx.genderx)

genderx,Female,Male
membershipCardx,,
No,29,24
Yes,30,30


El intervalo de confianza (IC) se construye usando dos entradas: la proporcion de la muestra de membership y el total sample size for yes-menbershipcard and non-menbershipcard combined. Calculamos estos valores a continuacion.

In [10]:
dz = dx.groupby(dx.genderx).agg({"membershipCardx": [lambda x: np.mean(x=="Yes"), np.size]})
dz.columns = ["Proportion", "Total_n"] # The default column names are unclear, so we replace them here
dz

,Proportion,Total_n
genderx,,
Female,0.508475,59
Male,0.555556,54


Con un intervalo de confianza del 95 %, se puede construir como el intervalo que consta de todos los puntos que estan dentro de dos (o 1,96) errores estandar de la estimacion puntual.

Dado que el error estandar juega un papel tan importante aqui, primero lo calculamos por separado.

In [11]:
p = dz.Proportion.Female # Female proportion
n = dz.Total_n.Female # Total number of females
se_female = np.sqrt(p * (1 - p) / n)
print(se_female)

p = dz.Proportion.Male # Male proportion
n = dz["Total_n"].Male # Total number of males
se_male = np.sqrt(p * (1 - p) / n)
print(se_male)

0.06508510485393613
0.06762006882779828


A continuacion, calculamos los intervalos de confianza del 95 % para las proporciones de membresia femenina y masculina utilizando la formula para el intervalo de confianza de una muestra para una proporcion:

In [12]:
p = dz.Proportion.Female # Female proportion
n = dz.Total_n.Female # Total number of females
lcb = p - 1.96 * np.sqrt(p * (1 - p) / n)  
ucb = p + 1.96 * np.sqrt(p * (1 - p) / n)  
print(lcb, ucb)

0.3809077707574716 0.6360413817849012


Los resultados anteriores indican que la proporcion de poblacion (para mujeres que tienen carta de membresia) puede estar entre 0.3809 y 0.636.

In [13]:
p = dz.Proportion.Male # Male proportion
n = dz.Total_n.Male # Total number of males
lcb = p - 1.96 * np.sqrt(p * (1 - p) / n)  
ucb = p + 1.96 * np.sqrt(p * (1 - p) / n)  
print(lcb, ucb)

0.423020220653071 0.6880908904580402


Para hombres que tienen carta de membresia proporción de población puede estar entre 0.423 y 0.688.

We can use the Statsmodels library to calculate the CI for us in one line:

In [14]:
# 95% CI for the proportion of females who membership (compare to value above)
sm.stats.proportion_confint(30, 29+30)

(0.3809101148274585, 0.6360390377149143)

In [15]:
# 95% CI for the proportion of males who membership (compare to value above)
sm.stats.proportion_confint(30, 24+30)

(0.42302265602095135, 0.6880884550901598)

# Confidence intervals comparing two independent proportions

A continuacion, calculamos la diferencia entre usuarios de membresia de la poblacion femenina con la masculina. Por lo tanto determinamos el error standar 

In [16]:
se_diff = np.sqrt(se_female**2 + se_male**2)
se_diff

0.0938538469223507

In [17]:
d = dz.Proportion.Female - dz.Proportion.Male
lcb = d - 2*se_diff
ucb = d + 2*se_diff
print(lcb, ucb)

-0.23478867312907056 0.14062671456033224


El intervalo va de -0.235 a 0.141, son valores cercanos a cero de tal manera que es posible que no haya diferencia entre estas dos poblaciones, es decir que la cantidad de usuarios que tienen carta de membresia es similar entre hombre y mujeres.

# Confidence intervals for the mean

Para fines didacticos en la siguiente seccion vamos a generar una columna aleatorio que contenga las edades de los encuestados tomando como base la codificacion dada por la encuesta.

Se contruira un intervalo de confianza de medias, para lograrlo nos preguntaremos ¿Cual es el promedio de edad de los consumidores que respondieron con 3 y 5 estrellas al servicio de las cafetarias Starbuks de Malaysia en el periodo correspondiente en que se corrio la encuesta?  

In [18]:
# usaremos la libreria random
from random import randrange

In [19]:
df["ageNumber"]=df.age.replace({0:randrange(15,19), 1:randrange(20,29), 2:randrange(30,39), 3:randrange(40,60)})

In [20]:
df["serviceRatex"]=df.serviceRate.replace({1:"Bad",2:"Bad",3:"Bad", 4:"Good", 5:"Good"})

Vamos a cruzar las variables califacion de servicio (serviceRatex) con ageNumber para conocer el promedio de edad segmentado por calificacion de servicio bueno y malo.

In [21]:
df.groupby("serviceRatex").agg({"ageNumber": np.mean})

,ageNumber
serviceRatex,
Bad,28.400000
Good,29.027397


Nuestros errores estandar para el rate bad y para rate good:

In [22]:
df.groupby("serviceRatex").agg({"ageNumber": [np.mean, np.std, np.size]})

ageNumber               
                   mean       std size
serviceRatex                          
Bad           28.400000  6.953324   40
Good          29.027397  5.720268   73

In [23]:
sem_bad = 7.897 / np.sqrt(40)
sem_good = 7.234 / np.sqrt(73)
print(sem_bad, sem_good)

1.2486253341174844 0.8466756588168086


Vamos a construir un intervalo de confianza de los clientes que clasificaron como bueno los servicios de la marca.

In [24]:
lcb_good = 25.92 - 1.96 * 7.234 / np.sqrt(73)
ucb_good = 25.92 + 1.96 * 7.234/ np.sqrt(73)
print(lcb_good, ucb_good)

24.26051570871906 27.579484291280945


Las personas que clasifican positivamente el servicio de la marca estan en una edad entre 24 a 27 años.

# Confidence intervals for the difference between two means

En esta seccion vamos a practicar la diferencia entre dos medias, para ello vamos a seguir con las variables edad y serviceRate. Nos vamos a auxiliar de los errores estandar creados anteriormente. 

In [25]:
sem_diff = np.sqrt(sem_bad**2 + sem_good**2)
sem_diff

1.5086168818599628

Construimos con un 95% de confianza el intervalo para la diferencia entre edad media para calificacion buena y mala: 

In [26]:
age_diff = 25.48 - 25.92
lcb = age_diff - 2*sem_diff
ucb = age_diff + 2*sem_diff
(lcb, ucb)

(-3.457233763719927, 2.5772337637199243)

La diferencia en la media de la edad que clasifica como bueno o malo el servicio esta entre -3 years a 2 years, indicandonos que es posible la no diferencia entre las edades debido a que se contempla el valor cero dentro del rango.